In [15]:
import numpy as np
import random

# Generation of Simulated Images

In [16]:
def generate_microscope_image(size):
  """
  Creates a binary image with a random, blob-like parasite occupying 25% or more of the area.
  Uses a combination of random walks and filling to create a more connected blob shape.
  """
  image = [0] * size**2

  # Ensure at least 25% of pixels are black (parasite)
  parasite_area = 0.3
  min_black_pixels = 0.3 * (size**2)
  black_pixels = 0

  # Start with a random seed pixel
  x, y = random.randint(0, size-1), random.randint(0, size-1)
  image[y * size + x] = 1
  black_pixels += 1

  # Perform a random walk to create a connected structure
  for _ in range(min_black_pixels // 2):
    # Choose a random direction (up, down, left, right)
    direction = random.choice([(0, 1), (0, -1), (1, 0), (-1, 0)])
    new_x, new_y = x + direction[0], y + direction[1]

    # Check if new position is within image bounds and not already black
    if 0 <= new_x < size and 0 <= new_y < size and image[new_y * size + new_x] == 0:
      x, y = new_x, new_y
      image[y * size + x] = 1
      black_pixels += 1

  # Fill any small gaps within the initial structure to improve blob-likeness
  fill_queue = [(x, y)]
  while fill_queue:
    x, y = fill_queue.pop(0)
    for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
      new_x, new_y = x + dx, y + dy
      if 0 <= new_x < size and 0 <= new_y < size and image[new_y * size + new_x] == 0:
        image[new_y * size + new_x] = 1
        black_pixels += 1
        fill_queue.append((new_x, new_y))

  # Add some noise for imperfections
  for _ in range(int(size**2 * 0.01)):
    x, y = random.randint(0, size-1), random.randint(0, size-1)
    image[y * size + x] = 1

  return image

In [17]:
# Run Length Encoding is used for microscopic image as we have a continuous area of black and white pixels

def encode_rle_black_only(image_data):
  """
  Generates RLE encoding from a binary image (list of 0s and 1s), considering only black pixels (value 1) run.
  """
  rle = []
  run_start = None  # Track the start of a black pixel run
  for i, pixel in enumerate(image_data):
    if pixel == 1:  # Black pixel encountered
      if run_start is None:
        run_start = i  # Start a new run if not already started
    else:  # Encountered a white pixel (or the beginning of the image)
      if run_start is not None:  # If a black run was in progress
        rle.append((run_start, i - run_start))  # Add the black run information
        run_start = None  # Reset run start for next black pixel

  # Add the last black run if it exists
  if run_start is not None:
    rle.append((run_start, len(image_data) - run_start))

  return rle

In [18]:
def decode_rle(rle_data, size):
  """
  Decodes RLE data (list of tuples) into a binary image (list of 0s and 1s).
  Assumes proper RLE format (tuples of starting position and run length).
  """
  image = [0]*(size**2)
  total_parasite_area = 0

  for start, run_length in rle_data:
    image[start: start+run_length] = 1
    total_parasite_area += run_length

  return image, total_parasite_area

In [19]:
# Sparse matrix representation is used for dye sensor images using dictionary as the lit pixels will be sparse

def generate_dye_sensor_image(size, sparsity=0.001):
  """
  Creates a sparse image with random luminescence points.
  """
  image = {}  # Use dictionary for sparse representation
  num_lit_pixels = int(size**2 * sparsity)
  for _ in range(num_lit_pixels):
    x, y = random.randint(0, size-1), random.randint(0, size-1)
    image[(y * size) + x] = 1
  return image

# Detection of Cancer

In [20]:
def has_cancer(microscope_image_RLE, dye_image, cancer_threshold=0.1):
  """
  Checks if a parasite has cancer based on dye intensity exceeding the threshold.
  """
  microscope_image, total_parasite_area = decode_rle(microscope_image_RLE)

  # Calculate total dye intensity within parasite region
  total_dye = 0

  for position, intensity in dye_image.items():
    if position < len(microscope_image) and microscope_image[position] == 1:  # Check if inside parasite
      total_dye += 1
      # Early stopping for efficiency (optional)
      if total_dye / total_parasite_area > cancer_threshold:
        return True

  # Determine if dye intensity exceeds cancer threshold
  return total_dye / total_parasite_area > cancer_threshold